# Scripted spectra

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
# T_final = 32000.
T_final = 32768.
tol = 1e-6
# τ_final = 2000.
τ_final = 2048.

npts_diag = 16
T_diags = T_final .* range(0.5, 1.0, npts_diag+1)
@sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/omegas", T_final, τ_final, T_diags[1] / T_final)

In [ ]:
all_mf_Es = []
all_exact_E0s = []
all_exact_E1s = []
all_ωs = []
spec_sum = []

num_plots = 100
for instance_name in instance_names#[1:num_plots]
    if occursin("results", instance_name)
        continue
    end
    try
        seed = match(pattern, instance_name)[1]    

        # mean-field solution and energy
        J_mat = h5read(folder_name * instance_name, "J")
        mf_problem = Problem(0, J_mat)

        mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
        sigma_star = sign.(mf_sol)
        h = mf_problem.local_fields
        J = mf_problem.couplings
        E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])
        push!(all_mf_Es, E)
        
        
        # gap and spectra
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        push!(all_exact_E0s, λ[1, end])
        push!(all_exact_E1s, λ[2, end])

        gap = λ[2, :] .- λ[1, :]    
        exact_times = range(0, 1, 33)
        gap_idx = findfirst(x -> x == minimum(gap), gap)
        gaploc = exact_times[gap_idx] 
        
        # T_diags = T_final .* [gaploc - 0.1, gaploc - 0.05, gaploc]

        gap_idx = gap_idx - 16 # we use only the interval [0.5, 1.0] with 16+1 points

        # for k in gap_idx:gap_idx
        for k in [gap_idx-2]
        # for k in [4]
            push!(all_ωs, h5read(folder_name * "results_" * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final)))
            push!(spec_sum, h5read(folder_name * "results_" * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final)))
        end   
    catch
        # printstyled(instance_name, "\n", color=:red)
    end
end

In [ ]:
num_plots = length(spec_sum)

In [ ]:
ΔE = all_mf_Es .- all_exact_E0s
ΔE1 = all_mf_Es .- all_exact_E1s
mean(ΔE) |> println
mean(ΔE1) |> println
findall(x -> x == 0, ΔE) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-1), ΔE) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-1), ΔE1) |> length |> println

In [ ]:
num_bins = 64
counts, bins = np.histogram(log.(abs.(ΔE) .+ 1.0im), bins=num_bins)
bins = real.(bins)

# PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["slategray", "midnightblue", "seagreen", "darkcyan"]))

figure(figsize=(4, 3))
hist(bins[1:end-1], bins, weights=counts/length(ΔE))
# legend(frameon=false)
xlim(0, 0.2)
ylim(0, 0.5)
xlabel("\$\\ln(E_* - E_0)\$")
ylabel("\$P(E_*)\$")
tight_layout()
# savefig(@sprintf("../plots/Histogram_MAX2SAT_L_%i.pdf", L))

In [ ]:
num_bins = 64
counts, bins = np.histogram(log.(abs.(ΔE1) .+ 1.0im), bins=num_bins)
bins = real.(bins)

# PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["slategray", "midnightblue", "seagreen", "darkcyan"]))

figure(figsize=(4, 3))
hist(bins[1:end-1], bins, weights=counts/length(ΔE1))
# legend(frameon=false)
xlim(0, 0.2)
ylim(0, 1.)
xlabel("\$\\ln(E_* - E_0)\$")
ylabel("\$P(E_*)\$")
tight_layout()
# savefig(@sprintf("../plots/Histogram_MAX2SAT_L_%i.pdf", L))

In [ ]:
# map data to functions
spectral_sum_fs = []
for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    push!(spectral_sum_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
xlim_vals = (-0.05, 0.4)
new_ωs = range(xlim_vals..., 2001)
spectral_average = mean([map(spectral_sum_fs[k], new_ωs) for k in 1:length(spectral_sum_fs)]);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
plot(new_ωs, spectral_average, "-k", ms=2)    
xlim(xlim_vals...)
ylim(-0., 0.04)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

In [ ]:
xlim_vals = (-0.05, 0.1)
fig, axs = subplots(figsize=(5, 50), nrows=num_plots, ncols=1)

for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    ax = axs[k]     
    ax.plot(ωs, spec, "-")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.05)  
    ax.set_xticks(range(xlim_vals..., 7))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    # ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 7), digits=3))

subplots_adjust(wspace=0.5, hspace=0.16)